In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt

In [ ]:
import numpy as np

from includes import utils
from includes import config
from includes import algorithms

In [ ]:
def evaluate_algorithm(train_set, test_set, algorithm, verbose, *args):
    if len(test_set) == 0:
        np.random.shuffle(train_set)
        
        train_set, test_set = np.split(train_set, [int(train_set.shape[0] * 0.7)])
        
    predicted, model = algorithm(train_set, test_set, verbose, *args)

    accuracy = np.mean(test_set[:, -1] == predicted) * 100

    return accuracy, model

In [ ]:
train_set, test_set, n_classes = utils.get_data("iris")

In [ ]:
conf = config.Config(verbose=False)

In [ ]:
conf.set_algorithm("sft-reg")
accuracy, model = evaluate_algorithm(
    train_set,
    test_set,
    algorithms.softmax_regression,
    conf.verbose,
    conf.lr,
    n_classes,
    conf.n_epochs,
    conf.batch_size,
    conf.tol
)

print "Softmax Regression"
print "Accuracy: %s" % accuracy

In [ ]:
conf.set_algorithm("boosted-sft-reg")

boosted_accuracies = []
boosted_models = []
for activation in ["logistic", "relu", "tanh", "identity"]:
    accuracy, model = evaluate_algorithm(
        train_set,
        test_set,
        algorithms.boosted_softmax_regression,
        conf.verbose,
        conf.lr,
        n_classes,
        conf.n_epochs,
        conf.batch_size,
        conf.m_stop,
        conf.eta,
        activation,
        "plots/sft-data-%s.npy" % activation
    )
    
    boosted_models.append(model)
    boosted_accuracies.append(accuracy)

    print "Boosted Softmax Regression, Activation = %s" % activation
    print "Final Test Accuracy: %f" % accuracy

In [ ]:
conf.set_algorithm("multi-mlp")

pretrained_mlp_models = []
pretrained_mlp_accuracies = []

activations = ["logistic", "relu", "tanh", "identity"]
for index, activation in enumerate(activations):
    accuracy, model = evaluate_algorithm(
        train_set,
        test_set,
        algorithms.pretrained_mlp,
        conf.verbose,
        conf.lr,
        n_classes,
        activation,
        conf.n_epochs,
        conf.batch_size,
        conf.tol,
        boosted_models[index]
    )
    
    pretrained_mlp_accuracies.append(accuracy)
    pretrained_mlp_models.append(model)

    print "Boosted + MLP, Activation = %s" % activation
    print "Final Test Accuracy: %f" % accuracy

In [ ]:
conf.set_algorithm("multi-mlp")

mlp_models = []
mlp_accuracies = []

activations = ["logistic", "relu", "tanh", "identity"]
for activation in activations:
    accuracy, model = evaluate_algorithm(
        train_set,
        test_set,
        algorithms.mlp,
        conf.verbose,
        conf.lr,
        n_classes,
        activation,
        conf.n_epochs,
        conf.batch_size,
        conf.hidden_size,
        conf.tol / 10
    )
    
    mlp_accuracies.append(accuracy)
    mlp_models.append(model)

    print "Multi Layer Perceptron, Activation = %s" % activation
    print "Accuracy: %f" % accuracy

In [ ]:
for i in range(4):
    print "%.6f  %.6f  %.6f" % (boosted_accuracies[i], mlp_accuracies[i], pretrained_mlp_accuracies[i])

In [ ]:
colors = ["blue", "red", "green", "orange"]
activations = ["logistic", "relu", "tanh", "identity"]

for color, activation in zip(colors, activations):
    data = np.load("plots/sft-data-%s.npy" % activation)
    plt.plot(data, label=activation, color=color)
    
plt.xlabel("m_stop")
plt.ylabel("test accuracy")
    
plt.legend()
plt.tight_layout()
plt.savefig("plots/plot.png", transparent=True)
plt.show()